# Manual Tree Editor
## Santa 2025 - Interactive Tree Placement Tool

**Features:**
- Card-based navigation by group n (1..200)
- Drag-and-release tree placement with collision rejection
- Rotation control with collision validation
- Persistent state (JSON) + CSV export
- Test mode (n ≤ 10)

In [ ]:
# Setup - Enable interactive matplotlib backend
%matplotlib widget

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon as MplPolygon
from matplotlib.collections import PatchCollection
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import os
import json
from datetime import datetime
from typing import Optional, Dict, List, Tuple

# Import our geometry module
from tree_geom_sat import (
    Placement, TREE, TW, TH, TREE_AREA,
    get_tree_at, check_collision, has_any_collision, find_collisions,
    check_single_collision, compute_side, compute_bounds, center_placements,
    load_submission_csv, save_submission_csv, load_state_json, save_state_json,
    compute_score, compute_group_score
)

print("Manual Tree Editor initialized!")
print(f"Tree dimensions: {TW:.4f} x {TH:.4f}")

In [ ]:
# Configuration
CONFIG = {
    'state_file': 'manual_state.json',
    'original_csv': '../submission.csv',  # Original submission to load from
    'output_csv': 'submission_manual.csv',
    'test_csv': 'submission_test_manual.csv',
    'max_n': 200,
    'test_max_n': 10,
    'autosave_interval': 30,  # seconds
}

print(f"Config: {CONFIG}")

In [ ]:
class TreeEditor:
    """
    Interactive tree placement editor with:
    - Drag and drop placement
    - Rotation control
    - Collision validation
    - Persistent state
    """

    def __init__(self, config: dict):
        self.config = config
        self.state: Dict[int, List[Placement]] = {}
        self.edit_timestamps: Dict[int, str] = {}
        self.current_n: int = 1
        self.selected_tree_idx: Optional[int] = None
        self.test_mode: bool = True
        self.last_valid_pose: Optional[Placement] = None

        # Drag state
        self.dragging = False
        self.drag_start_pos = None

        # UI components
        self.fig = None
        self.ax = None
        self.tree_patches = []
        self.status_label = None

        # Load state
        self._load_state()

    def _load_state(self):
        """Load state from JSON or CSV."""
        state_file = self.config['state_file']
        csv_file = self.config['original_csv']

        if os.path.exists(state_file):
            print(f"Loading state from {state_file}...")
            self.state = load_state_json(state_file)
            # Load timestamps if available
            ts_file = state_file.replace('.json', '_timestamps.json')
            if os.path.exists(ts_file):
                with open(ts_file, 'r') as f:
                    self.edit_timestamps = json.load(f)
            print(f"Loaded {len(self.state)} groups from state file")
        elif os.path.exists(csv_file):
            print(f"Loading state from {csv_file}...")
            self.state = load_submission_csv(csv_file)
            print(f"Loaded {len(self.state)} groups from CSV")
        else:
            print("No existing state found. Starting fresh.")
            # Initialize with empty state (will be filled on demand)
            pass

    def save_state(self):
        """Save current state to JSON."""
        save_state_json(self.state, self.config['state_file'])
        # Save timestamps
        ts_file = self.config['state_file'].replace('.json', '_timestamps.json')
        with open(ts_file, 'w') as f:
            json.dump(self.edit_timestamps, f, indent=2)
        self._update_status(f"State saved at {datetime.now().strftime('%H:%M:%S')}")

    def export_csv(self, test_only: bool = False) -> bool:
        """Export state to CSV file."""
        # Validate all groups first
        invalid_groups = []
        for n, placements in self.state.items():
            if has_any_collision(placements):
                invalid_groups.append(n)

        if invalid_groups:
            self._update_status(f"EXPORT BLOCKED: Groups {invalid_groups[:5]}... have collisions!", error=True)
            return False

        if test_only:
            # Export only n <= test_max_n
            export_state = {n: p for n, p in self.state.items() if n <= self.config['test_max_n']}
            filepath = self.config['test_csv']
        else:
            export_state = self.state
            filepath = self.config['output_csv']

        save_submission_csv(export_state, filepath)

        # Calculate statistics
        total_rows = sum(len(p) for p in export_state.values())
        score = compute_score(export_state, max(export_state.keys()) if export_state else 0)

        self._update_status(f"Exported {total_rows} rows to {filepath} (score: {score:.2f})")
        return True

    def _update_status(self, message: str, error: bool = False):
        """Update status label."""
        if self.status_label:
            color = 'red' if error else 'black'
            self.status_label.value = f'<span style="color:{color}">{message}</span>'

    def get_group_info(self, n: int) -> dict:
        """Get info for a group."""
        if n not in self.state:
            return {'n': n, 'overlap_count': 0, 'side': 0.0, 'score': 0.0, 'last_edited': 'Never'}

        placements = self.state[n]
        collisions = find_collisions(placements)
        side = compute_side(placements)
        score = compute_group_score(placements, n)
        last_edited = self.edit_timestamps.get(str(n), 'Never')

        return {
            'n': n,
            'overlap_count': len(collisions),
            'side': side,
            'score': score,
            'last_edited': last_edited
        }

    def select_group(self, n: int):
        """Select a group for editing."""
        self.current_n = n
        self.selected_tree_idx = None
        self.last_valid_pose = None
        self._redraw_canvas()

    def _get_current_placements(self) -> List[Placement]:
        """Get placements for current group."""
        return self.state.get(self.current_n, [])

    def _redraw_canvas(self):
        """Redraw the canvas with current placements."""
        if self.ax is None:
            return

        self.ax.clear()
        placements = self._get_current_placements()

        if not placements:
            self.ax.set_title(f"Group n={self.current_n} - No data")
            self.ax.set_xlim(-5, 5)
            self.ax.set_ylim(-5, 5)
            self.ax.set_aspect('equal')
            self.ax.grid(True, alpha=0.3)
            self.fig.canvas.draw_idle()
            return

        # Calculate bounds for display
        bounds = compute_bounds(placements)
        margin = max(TW, TH) * 0.5

        # Draw each tree
        self.tree_patches = []
        for i, p in enumerate(placements):
            vertices = get_tree_at(p)

            # Color based on selection and collisions
            if i == self.selected_tree_idx:
                color = 'orange'
                alpha = 0.9
                zorder = 10
            elif check_single_collision(placements, i):
                color = 'red'
                alpha = 0.7
                zorder = 5
            else:
                color = 'green'
                alpha = 0.6
                zorder = 1

            patch = MplPolygon(vertices, closed=True, facecolor=color,
                               edgecolor='black', linewidth=1, alpha=alpha, zorder=zorder)
            self.ax.add_patch(patch)
            self.tree_patches.append(patch)

            # Add tree index label
            self.ax.annotate(str(i), (p.x, p.y), fontsize=8, ha='center', va='center',
                           color='white', fontweight='bold', zorder=zorder+1)

        # Set view limits
        self.ax.set_xlim(bounds[0] - margin, bounds[2] + margin)
        self.ax.set_ylim(bounds[1] - margin, bounds[3] + margin)
        self.ax.set_aspect('equal')
        self.ax.grid(True, alpha=0.3)

        # Title with info
        side = compute_side(placements)
        score = compute_group_score(placements, self.current_n)
        collisions = find_collisions(placements)
        status = "OK" if len(collisions) == 0 else f"{len(collisions)} overlaps!"
        sel_info = f" | Selected: {self.selected_tree_idx}" if self.selected_tree_idx is not None else ""
        self.ax.set_title(f"Group n={self.current_n} | Side: {side:.4f} | Score: {score:.4f} | {status}{sel_info}")

        self.fig.canvas.draw_idle()

    def _find_tree_at_point(self, x: float, y: float) -> Optional[int]:
        """Find tree index at given point."""
        placements = self._get_current_placements()
        if not placements:
            return None

        # Check from top to bottom (last drawn = highest z-order = selected first)
        order = list(range(len(placements)))
        if self.selected_tree_idx is not None:
            order.remove(self.selected_tree_idx)
            order.append(self.selected_tree_idx)

        for i in reversed(order):
            vertices = get_tree_at(placements[i])
            if self._point_in_polygon(x, y, vertices):
                return i

        return None

    def _point_in_polygon(self, x: float, y: float, vertices: np.ndarray) -> bool:
        """Check if point is inside polygon using ray casting."""
        n = len(vertices)
        inside = False
        j = n - 1
        for i in range(n):
            xi, yi = vertices[i]
            xj, yj = vertices[j]
            if ((yi > y) != (yj > y)) and (x < (xj - xi) * (y - yi) / (yj - yi) + xi):
                inside = not inside
            j = i
        return inside

    def _on_mouse_press(self, event):
        """Handle mouse press."""
        if event.inaxes != self.ax:
            return

        tree_idx = self._find_tree_at_point(event.xdata, event.ydata)

        if tree_idx is not None:
            self.selected_tree_idx = tree_idx
            placements = self._get_current_placements()
            self.last_valid_pose = placements[tree_idx].copy()
            self.dragging = True
            self.drag_start_pos = (event.xdata, event.ydata)
            self._update_rotation_slider()
        else:
            self.selected_tree_idx = None
            self.last_valid_pose = None
            self._update_rotation_slider()

        self._redraw_canvas()

    def _on_mouse_move(self, event):
        """Handle mouse move (drag preview)."""
        if not self.dragging or event.inaxes != self.ax:
            return
        if self.selected_tree_idx is None or self.last_valid_pose is None:
            return

        # Calculate new position
        dx = event.xdata - self.drag_start_pos[0]
        dy = event.ydata - self.drag_start_pos[1]

        # Update position temporarily for preview
        placements = self._get_current_placements()
        placements[self.selected_tree_idx].x = self.last_valid_pose.x + dx
        placements[self.selected_tree_idx].y = self.last_valid_pose.y + dy

        self._redraw_canvas()

    def _on_mouse_release(self, event):
        """Handle mouse release - validate and commit or revert."""
        if not self.dragging:
            return

        self.dragging = False

        if self.selected_tree_idx is None or self.last_valid_pose is None:
            return

        placements = self._get_current_placements()
        idx = self.selected_tree_idx

        # Check for collisions
        if check_single_collision(placements, idx):
            # Revert to last valid position
            placements[idx].x = self.last_valid_pose.x
            placements[idx].y = self.last_valid_pose.y
            self._update_status("Move rejected: overlap detected", error=True)
        else:
            # Commit the move
            self.last_valid_pose = placements[idx].copy()
            self.edit_timestamps[str(self.current_n)] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            self._update_status(f"Tree {idx} moved to ({placements[idx].x:.4f}, {placements[idx].y:.4f})")

        self._redraw_canvas()

    def apply_rotation(self, angle: float) -> bool:
        """Apply rotation to selected tree (validate before commit)."""
        if self.selected_tree_idx is None:
            self._update_status("No tree selected", error=True)
            return False

        placements = self._get_current_placements()
        idx = self.selected_tree_idx
        old_deg = placements[idx].deg

        # Try new rotation
        placements[idx].deg = angle % 360

        if check_single_collision(placements, idx):
            # Revert
            placements[idx].deg = old_deg
            self._update_status(f"Rotation rejected: overlap detected at {angle:.1f}deg", error=True)
            return False
        else:
            # Commit
            if self.last_valid_pose:
                self.last_valid_pose.deg = placements[idx].deg
            self.edit_timestamps[str(self.current_n)] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            self._update_status(f"Tree {idx} rotated to {placements[idx].deg:.1f} deg")
            self._redraw_canvas()
            return True

    def nudge_tree(self, dx: float, dy: float) -> bool:
        """Nudge selected tree by small amount."""
        if self.selected_tree_idx is None:
            self._update_status("No tree selected", error=True)
            return False

        placements = self._get_current_placements()
        idx = self.selected_tree_idx
        old_x, old_y = placements[idx].x, placements[idx].y

        # Try new position
        placements[idx].x += dx
        placements[idx].y += dy

        if check_single_collision(placements, idx):
            # Revert
            placements[idx].x = old_x
            placements[idx].y = old_y
            self._update_status(f"Nudge rejected: overlap", error=True)
            return False
        else:
            # Commit
            if self.last_valid_pose:
                self.last_valid_pose.x = placements[idx].x
                self.last_valid_pose.y = placements[idx].y
            self.edit_timestamps[str(self.current_n)] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            self._update_status(f"Tree {idx} nudged to ({placements[idx].x:.4f}, {placements[idx].y:.4f})")
            self._redraw_canvas()
            return True

    def reset_group(self, n: int):
        """Reset a group from original CSV."""
        csv_file = self.config['original_csv']
        if not os.path.exists(csv_file):
            self._update_status(f"Original CSV not found: {csv_file}", error=True)
            return

        original = load_submission_csv(csv_file)
        if n in original:
            self.state[n] = original[n]
            self.edit_timestamps[str(n)] = datetime.now().strftime('%Y-%m-%d %H:%M:%S') + ' (reset)'
            self._update_status(f"Group {n} reset from original CSV")
            if n == self.current_n:
                self._redraw_canvas()
        else:
            self._update_status(f"Group {n} not found in original CSV", error=True)

    def _update_rotation_slider(self):
        """Update rotation slider to match selected tree."""
        if hasattr(self, 'rotation_slider') and self.rotation_slider:
            if self.selected_tree_idx is not None:
                placements = self._get_current_placements()
                self.rotation_slider.value = placements[self.selected_tree_idx].deg

    def build_ui(self):
        """Build the complete UI."""
        # Create figure
        self.fig, self.ax = plt.subplots(figsize=(10, 8))
        self.fig.canvas.toolbar_visible = True
        self.fig.canvas.header_visible = False

        # Connect mouse events
        self.fig.canvas.mpl_connect('button_press_event', self._on_mouse_press)
        self.fig.canvas.mpl_connect('motion_notify_event', self._on_mouse_move)
        self.fig.canvas.mpl_connect('button_release_event', self._on_mouse_release)

        # Status label
        self.status_label = widgets.HTML(value='Ready')

        # Test mode toggle
        test_mode_toggle = widgets.Checkbox(value=self.test_mode, description='Test Mode (n≤10)')

        def on_test_mode_change(change):
            self.test_mode = change['new']
            update_cards()

        test_mode_toggle.observe(on_test_mode_change, names='value')

        # Group selector
        max_n = self.config['test_max_n'] if self.test_mode else self.config['max_n']
        group_selector = widgets.IntSlider(value=1, min=1, max=max_n, description='Group n:')

        def on_group_change(change):
            self.select_group(change['new'])
            update_card_highlight(change['new'])

        group_selector.observe(on_group_change, names='value')

        # Rotation slider (continuous_update=False for performance)
        self.rotation_slider = widgets.FloatSlider(
            value=0, min=0, max=360, step=0.5,
            description='Rotation:',
            continuous_update=False
        )

        def on_rotation_change(change):
            self.apply_rotation(change['new'])

        self.rotation_slider.observe(on_rotation_change, names='value')

        # Rotation fine-tune buttons
        def make_rotation_button(delta, label):
            btn = widgets.Button(description=label, layout=widgets.Layout(width='60px'))
            def on_click(b):
                if self.selected_tree_idx is not None:
                    placements = self._get_current_placements()
                    new_angle = (placements[self.selected_tree_idx].deg + delta) % 360
                    self.apply_rotation(new_angle)
                    self.rotation_slider.value = new_angle
            btn.on_click(on_click)
            return btn

        rotation_buttons = widgets.HBox([
            make_rotation_button(-5, '-5deg'),
            make_rotation_button(-1, '-1deg'),
            make_rotation_button(-0.5, '-0.5deg'),
            make_rotation_button(0.5, '+0.5deg'),
            make_rotation_button(1, '+1deg'),
            make_rotation_button(5, '+5deg'),
        ])

        # Nudge buttons
        nudge_step = 0.01

        def make_nudge_button(dx, dy, label):
            btn = widgets.Button(description=label, layout=widgets.Layout(width='50px'))
            def on_click(b):
                self.nudge_tree(dx, dy)
            btn.on_click(on_click)
            return btn

        nudge_buttons = widgets.VBox([
            widgets.HBox([widgets.Label('Nudge:'), make_nudge_button(0, nudge_step, 'Up')]),
            widgets.HBox([
                make_nudge_button(-nudge_step, 0, 'Left'),
                make_nudge_button(nudge_step, 0, 'Right')
            ]),
            widgets.HBox([widgets.Label(''), make_nudge_button(0, -nudge_step, 'Down')]),
        ])

        # Save button
        save_btn = widgets.Button(description='Save State', button_style='primary',
                                  layout=widgets.Layout(width='120px'))
        save_btn.on_click(lambda b: self.save_state())

        # Export buttons
        export_btn = widgets.Button(description='Export Full CSV', button_style='success',
                                   layout=widgets.Layout(width='140px'))
        export_btn.on_click(lambda b: self.export_csv(test_only=False))

        export_test_btn = widgets.Button(description='Export Test CSV', button_style='info',
                                        layout=widgets.Layout(width='140px'))
        export_test_btn.on_click(lambda b: self.export_csv(test_only=True))

        # Reset group button
        reset_btn = widgets.Button(description='Reset Group', button_style='warning',
                                   layout=widgets.Layout(width='120px'))
        reset_btn.on_click(lambda b: self.reset_group(self.current_n))

        # Cards list (scrollable)
        cards_output = widgets.Output(layout=widgets.Layout(
            height='400px', overflow_y='scroll', border='1px solid #ccc'
        ))

        def update_cards():
            """Update cards list."""
            max_n = self.config['test_max_n'] if self.test_mode else self.config['max_n']
            group_selector.max = max_n

            with cards_output:
                clear_output(wait=True)
                for n in range(1, max_n + 1):
                    info = self.get_group_info(n)
                    status_color = 'green' if info['overlap_count'] == 0 else 'red'
                    highlight = 'background-color: #e0e0ff;' if n == self.current_n else ''

                    html = f'''
                    <div style="padding: 5px; margin: 2px; border: 1px solid #ccc; cursor: pointer; {highlight}">
                        <b>n={n}</b> |
                        <span style="color:{status_color}">Overlaps: {info['overlap_count']}</span> |
                        Side: {info['side']:.4f} |
                        Score: {info['score']:.4f}<br>
                        <small>Last: {info['last_edited']}</small>
                    </div>
                    '''
                    display(HTML(html))

        def update_card_highlight(n):
            """Refresh cards to update highlight."""
            update_cards()

        # Score display
        score_output = widgets.Output()

        def update_score():
            with score_output:
                clear_output(wait=True)
                max_n = self.config['test_max_n'] if self.test_mode else self.config['max_n']
                score = compute_score(self.state, max_n)
                print(f"Total Score (n=1..{max_n}): {score:.2f}")

        # Refresh button
        refresh_btn = widgets.Button(description='Refresh Cards', layout=widgets.Layout(width='120px'))
        refresh_btn.on_click(lambda b: (update_cards(), update_score()))

        # Layout
        controls = widgets.VBox([
            widgets.HTML('<h4>Controls</h4>'),
            test_mode_toggle,
            group_selector,
            widgets.HTML('<hr>'),
            widgets.HTML('<b>Rotation</b>'),
            self.rotation_slider,
            rotation_buttons,
            widgets.HTML('<hr>'),
            nudge_buttons,
            widgets.HTML('<hr>'),
            widgets.HBox([save_btn, reset_btn]),
            widgets.HBox([export_btn, export_test_btn]),
            refresh_btn,
            widgets.HTML('<hr>'),
            score_output,
        ])

        cards_panel = widgets.VBox([
            widgets.HTML('<h4>Groups</h4>'),
            cards_output
        ])

        left_panel = widgets.VBox([cards_panel, controls],
                                   layout=widgets.Layout(width='300px'))

        right_panel = widgets.VBox([
            self.status_label,
            self.fig.canvas
        ])

        main_layout = widgets.HBox([left_panel, right_panel])

        # Initial draw
        update_cards()
        update_score()
        self.select_group(1)

        return main_layout

In [ ]:
# Create and run the editor
editor = TreeEditor(CONFIG)
ui = editor.build_ui()
display(ui)

## Usage Instructions

### Selecting and Moving Trees
1. **Click on a tree** to select it (turns orange)
2. **Drag and release** to move it
3. If the new position causes overlap, the move is rejected and the tree snaps back

### Rotating Trees
1. Select a tree first
2. Use the **rotation slider** or **fine-tune buttons** (±0.5°, ±1°, ±5°)
3. Rotation changes are validated on release - invalid rotations are reverted

### Nudging Trees
- Use the **Up/Down/Left/Right** buttons for precise 0.01 unit movements
- Invalid nudges are rejected

### Switching Groups
- Use the **Group n slider** or click a card in the left panel
- Edits are preserved when switching groups

### Saving and Exporting
- **Save State**: Saves to `manual_state.json` for persistence across sessions
- **Export Full CSV**: Creates `submission_manual.csv` with all 200 groups
- **Export Test CSV**: Creates `submission_test_manual.csv` with n≤10 only
- **Reset Group**: Reverts current group to original CSV values

### Test Mode
- Toggle **Test Mode** to show only groups n=1..10
- Useful for quick testing without loading all 200 groups

In [ ]:
# Manual save (run this cell to force save)
editor.save_state()
print("State saved!")

In [ ]:
# Validation check - run this to verify all groups are collision-free
print("Validating all groups...")
invalid_groups = []
for n in range(1, CONFIG['max_n'] + 1):
    if n in editor.state:
        if has_any_collision(editor.state[n]):
            collisions = find_collisions(editor.state[n])
            invalid_groups.append((n, len(collisions)))
            print(f"  n={n}: {len(collisions)} collision pairs")

if invalid_groups:
    print(f"\nWARNING: {len(invalid_groups)} groups have collisions!")
else:
    print("\nAll groups are collision-free!")

# Calculate total score
score = compute_score(editor.state)
print(f"\nTotal Score: {score:.2f}")

In [ ]:
# Export final submission
if editor.export_csv(test_only=False):
    print(f"Successfully exported to {CONFIG['output_csv']}")
else:
    print("Export failed - check for collisions!")